https://stepik.org/lesson/309343/step/4

In [ ]:
import torch
from abc import ABC, abstractmethod


# абстрактный класс для сверточного слоя
class ABCConv2d(ABC):
    # инициализирует основные параметры свёртки
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        # количество каналов на входе свёрточного слоя. Для RGB будет = 3
        self.in_channels = in_channels
        # количество каналов на выходе свёрточного слоя. Для RGB будет = 3
        self.out_channels = out_channels
        # размер ядра свёртки, если = 3, то ядро свёртки будет размером 3x3
        self.kernel_size = kernel_size
        # Шаг свёртки
        self.stride = stride

    # метод позволяет задавать весовые коэффициенты ядра свёртки
    # ниже в дочернем классе Conv2d мы его переопределим
    def set_kernel(self, kernel):
        self.kernel = kernel
    '''
    Метод __call__ в Python — это специальный метод, который позволяет экземплярам класса вести себя как функции.
    Если класс определяет метод __call__, вы можете вызывать объекты этого класса так же, как и обычные функции.
    '''
    # в нашем абстрактном классе ABCConv2d метод __call__ объявлен как абстрактный метод:
    @abstractmethod
    # значит любые подклассы, наследующиеся от ABCConv2d, обязаны реализовать этот метод.
    # в конкретных реализациях, таких как наш класс Conv2d, метод __call__ будет выполнять свёртку над входным тензором.
    def __call__(self, input_tensor):
        pass


# Conv2d - класс-обёртка над torch.nn.Conv2d для унификации интерфейса
# класс реализует конкретный свёрточный слой, используя torch.nn.Conv2d из PyTorch
class Conv2d(ABCConv2d):
    # Конструктор создаёт объект torch.nn.Conv2d, который будет выполнять свёртку
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)
    # задаёт веса ядра свёртки
    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        # реализует свёртку, вызывая свёрточный слой на входном тензоре
        return self.conv2d(input_tensor)


# функция:
def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    # извлекает нужные размеры из ядра
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]
    # создаёт экземпляр класса свёрточного слоя conv2d_layer_class
    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    # устанавливает ядро свёртки (методом из астрактного класса)
    layer.set_kernel(kernel)
    # выполняет свёртку над входным тензором input_matrix и возвращает результат
    return layer(input_matrix)


# функция, тестирующая класс conv2d_cls
# возвращает True, если свертка совпадает со сверткой с помощью torch.nn.Conv2d.
def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    # создаёт ядро свёртки
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    # подсчитывает количество каналов в ядре
    in_channels = kernel.shape[1]

    # создаёт входной тензор
    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    # вызывает функцию для создания и выполнения свёртки
    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    # сравнивает результат с эталонной реализацией на основе Conv2d
    # возвращает True, если результаты и размеры совпадают
    return torch.allclose(custom_conv2d_out, conv2d_out) \
        and (custom_conv2d_out.shape == conv2d_out.shape)

# вызов запускает тест и выводит True, если свёртка, выполненная классом Conv2d, работает корректно
print(test_conv2d_layer(Conv2d))

In [ ]:
#@title кусочек кода, генерирующий входной тензор:
import torch

batch_size=2
in_channels=3
input_height=3
input_width=3

input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

input_tensor

tensor([[[[ 0.,  1.,  2.],
          [ 3.,  4.,  5.],
          [ 6.,  7.,  8.]],

         [[ 9., 10., 11.],
          [12., 13., 14.],
          [15., 16., 17.]],

         [[18., 19., 20.],
          [21., 22., 23.],
          [24., 25., 26.]]],


        [[[27., 28., 29.],
          [30., 31., 32.],
          [33., 34., 35.]],

         [[36., 37., 38.],
          [39., 40., 41.],
          [42., 43., 44.]],

         [[45., 46., 47.],
          [48., 49., 50.],
          [51., 52., 53.]]]])

In [ ]:
#@title комментарии от Oleg V: https://stepik.org/lesson/309343/step/4?discussion=1931534&unit=291492
import torch
from abc import ABC, abstractmethod


# абстрактный класс для сверточного слоя
class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel
## ниже указан декоратор @abstractmethod. Это значит, что при создании
## нового класса на основании этого функцию нужно будет переопределить
## иначе будет ошибка
    @abstractmethod
    def __call__(self, input_tensor):
        pass


# класс-обертка над torch.nn.Conv2d для унификации интерфейса
## Это класс который выполняет свёртку двумерного слоя
## согласно документации https://pytorch.org/docs/stable/nn.html#conv2d
## выполняется это следующим образом. Сам conv2d это класс.
## Для начала нужно передать ему параметры. После этого вызвать как функцию
## с матрицей, которую нужно свернуть.
## Не смог найти в документации это - self.conv2d.weight.data = kernel
class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)



    # функция, создающая объект класса cls и возвращающая свертку от input_matrix
## Эта функия подготоавливает данные и вызывает нужный класс для свертки
## Сделано это для упрощения.
## Обратите внимание самым первым параметром указывается
## класс, которым нужно свернуть матрицу   conv2d_layer_class

def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


# Функция, тестирующая класс conv2d_cls.
## Возвращает True, если свертка совпадает со сверткой с помощью torch.nn.Conv2d.
## Функция тестирует conv2d_layer_class.

def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    print('Вызвана функция test_conv2d_layer ей передан на тестирование класс ',conv2d_layer_class)

    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])
    print('Kernel с помощю которого будет выполняться сворачивание \n', kernel)
    in_channels = kernel.shape[1]

## создается тензор размерности 6 таблиц 4*4
## (то есть высота- 4, ширина -4) количество каналов - 3 (берется из размера kernel)
## и 2 батча 2*3=6
    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()).reshape(batch_size, in_channels, input_height, input_width)
    print('Матрица, которую надо будет свернуть \n', input_tensor)
 ## Здесь вызываются 2 фукнции, которые выполняют сворачивание
 ## Одной передается класс conv2d_layer_class другой Conv2d
 ## Обратите внимание что conv2d_layer_class это аргумент текущей функции
 ## Эту функцию вызвали через print(test_conv2d_layer(Conv2d))
 ## то есть по факту сравнивается Conv2d и Conv2d

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    print('Класс ',conv2d_layer_class,' выдает свернутую матрицу \n',custom_conv2d_out)

    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)
    print('Класс ',Conv2d,' выдает свернутую матрицу \n',conv2d_out)

    return torch.allclose(custom_conv2d_out, conv2d_out) and (custom_conv2d_out.shape == conv2d_out.shape)

print(test_conv2d_layer(Conv2d))

Вызвана функция test_conv2d_layer ей передан на тестирование класс  <class '__main__.Conv2d'>
Kernel с помощю которого будет выполняться сворачивание 
 tensor([[[[ 0.,  1.,  0.],
          [ 1.,  2.,  1.],
          [ 0.,  1.,  0.]],

         [[ 1.,  2.,  1.],
          [ 0.,  3.,  3.],
          [ 0.,  1., 10.]],

         [[10., 11., 12.],
          [13., 14., 15.],
          [16., 17., 18.]]]])
Матрица, которую надо будет свернуть 
 tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[16., 17., 18., 19.],
          [20., 21., 22., 23.],
          [24., 25., 26., 27.],
          [28., 29., 30., 31.]],

         [[32., 33., 34., 35.],
          [36., 37., 38., 39.],
          [40., 41., 42., 43.],
          [44., 45., 46., 47.]]],


        [[[48., 49., 50., 51.],
          [52., 53., 54., 55.],
          [56., 57., 58., 59.],
          [60., 61., 62., 63.]],

         [[64., 65., 66., 67.],
     